In [1]:
import proteusPy

from proteusPy import (
    DisulfideList,
    Disulfide,
    Load_PDB_SS,
    load_disulfides_from_id,
    prune_extra_ss,
    check_header_from_file,
    Vector3D,
)

from proteusPy.logger_config import get_logger
from proteusPy.ssparser import (
    extract_ssbonds_and_atoms,
    print_disulfide_bond_info_dict,
    get_phipsi_atoms_coordinates,
)
import numpy as np


_logger = get_logger("testing")

ProteusPy V0.97.4.20240828dev


In [2]:
# Load the PDB file
pdb = Load_PDB_SS(verbose=True, subset=False)
pdb.describe()

-> load_PDB_SS(): Reading /Users/egs/miniforge3/envs/ppydev/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
-> load_PDB_SS(): Done reading /Users/egs/miniforge3/envs/ppydev/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
    =========== RCSB Disulfide Database Summary ==============
       =========== Built: 2024-08-13 10:32:49 ==============
PDB IDs present:                    37075
Disulfides loaded:                  175649
Average structure resolution:       2.07 Å
Lowest Energy Disulfide:            2q7q_75D_140D
Highest Energy Disulfide:           6vxk_801B_806B
Cα distance cutoff:                 -1.00 Å
Total RAM Used:                     44.35 GB.
    ================= proteusPy: 0.97.0dev9 =======================


In [ ]:
pdb[0]
sslist = pdb.SSList
len(sslist)
sslist.minmax_energy

In [ ]:
slist = DisulfideList(pdb[0:10000], "tst")

In [ ]:
sslist.distance_df

In [ ]:
import logging

logger = logging.getLogger("proteusPy.Disulfide")
logger.setLevel(logging.INFO)
logger.info("test")

In [5]:
import logging


def set_logger_level_for_module(pkg_name, level=""):
    logger_dict = logging.Logger.manager.loggerDict
    registered_loggers = [
        name
        for name in logger_dict
        if isinstance(logger_dict[name], logging.Logger) and name.startswith(pkg_name)
    ]
    for logger_name in registered_loggers:
        logger = logging.getLogger(logger_name)
        if level:
            logger.setLevel(level)

    return registered_loggers


# Example usage
pkg_name = "proteusPy"
registered_loggers = set_logger_level_for_module(pkg_name, level=logging.DEBUG)
print(f"Registered loggers for '{pkg_name}':", registered_loggers)

Registered loggers for 'proteusPy': ['proteusPy.utility', 'proteusPy.vector3D', 'proteusPy.DisulfideList', 'proteusPy.Disulfide', 'proteusPy.DisulfideClass_Constructor', 'proteusPy.DisulfideLoader', 'proteusPy.ssparser']


In [ ]:
import proteusPy.vector3D

In [ ]:
pdb[0]

In [ ]:
ss1 = Disulfide()
ss1.Tor

In [ ]:
# Example usage:
ssbond_dict, num_ssbonds, errors = extract_ssbonds_and_atoms(
    "/Users/egs/PDB/good/pdb6f99.ent"
)

chain_id = "A"
key = "proximal-1"
phipsi_atoms = get_phipsi_atoms(ssbond_dict, chain_id, key)
print(phipsi_atoms)
print_disulfide_bond_info_dict(ssbond_dict)

In [ ]:
import os


def print_disulfide_bond_info_dict(ssbond_atom_data):
    """
    Prints the disulfide bond information in a pretty format.

    Args:
    - ssbond_atom_data (dict): A dictionary containing the SSBOND records and the corresponding ATOM records. The dictionary
          has the following structure:
            {
                "ssbonds": list of SSBOND records (str),
                "atoms": {
                    (chain_id, res_seq_num, atom_name): {
                        "x": x-coordinate (float),
                        "y": y-coordinate (float),
                        "z": z-coordinate (float)
                    },
                    ...
                },
                "pairs": [
                    {
                        "proximal": (chain_id1, res_seq_num1),
                        "distal": (chain_id2, res_seq_num2),
                        "chains": (chain_id1, chain_id2),
                        "phipsi": {
                            "proximal-1": {"N": [x, y, z], "C": [x, y, z]},
                            "proximal+1": {"N": [x, y, z], "C": [x, y, z]},
                            "distal-1": {"N": [x, y, z], "C": [x, y, z]},
                            "distal+1": {"N": [x, y, z], "C": [x, y, z]}
                        }
                    },
                    ...
                ]
            }
    """
    if ssbond_atom_list is None:
        print("No disulfide bonds found.")
        return

    ssbonds = ssbond_atom_data.get("ssbonds", [])
    atoms = ssbond_atom_data.get("atoms", {})
    pairs = ssbond_atom_data.get("pairs", [])

    for pair in pairs:
        proximal = pair["proximal"]
        distal = pair["distal"]
        chain_id1, res_seq_num1 = proximal
        chain_id2, res_seq_num2 = distal

        print(
            f"Disulfide Bond between Chain {chain_id1} Residue {res_seq_num1} and Chain {chain_id2} Residue {res_seq_num2}"
        )
        print(f"Proximal Residue (Chain {chain_id1}, Residue {res_seq_num1}):")
        for atom_name in ["N", "CA", "C", "O", "CB", "SG"]:
            atom_record = atoms.get((chain_id1, res_seq_num1, atom_name))
            if atom_record:
                print(
                    f"  Atom {atom_name}: ({atom_record['x']:.3f}, {atom_record['y']:.3f}, {atom_record['z']:.3f})"
                )
            else:
                print(f"  Atom {atom_name}: Not found")

        print(f"Distal Residue (Chain {chain_id2}, Residue {res_seq_num2}):")
        for atom_name in ["N", "CA", "C", "O", "CB", "SG"]:
            atom_record = atoms.get((chain_id2, res_seq_num2, atom_name))
            if atom_record:
                print(
                    f"  Atom {atom_name}: ({atom_record['x']:.3f}, {atom_record['y']:.3f}, {atom_record['z']:.3f})"
                )
            else:
                print(f"  Atom {atom_name}: Not found")

        print("Phi/Psi Atoms:")
        for key, phipsi_atoms in pair["phipsi"].items():
            print(f"  {key}:")
            for atom_name, coords in phipsi_atoms.items():
                res_seq_num = (
                    int(res_seq_num1) - 1
                    if "proximal-1" in key
                    else (
                        int(res_seq_num1) + 1
                        if "proximal+1" in key
                        else (
                            int(res_seq_num2) - 1
                            if "distal-1" in key
                            else int(res_seq_num2) + 1
                        )
                    )
                )
                print(
                    f"    Atom {atom_name} (Residue {res_seq_num}): ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
                )

        print("-" * 50)


# Example usage:
ssbond_dict, num_ssbonds, errors = extract_ssbonds_and_atoms(
    "/Users/egs/PDB/good/pdb6f99.ent"
)

print_disulfide_bond_info(ssbond_dict)

In [ ]:
def extract_ssbonds_and_atoms(input_pdb_file, verbose=False) -> tuple:
    """
    Extracts SSBOND and ATOM records from a PDB file.

    This function reads a PDB file to collect SSBOND records and ATOM records for cysteine residues.
    It then extracts the ATOM records corresponding to the SSBOND records and returns the collected
    data as a dictionary, along with the number of SSBOND records found and any errors encountered.

    Args:
    - input_pdb_file (str): The path to the input PDB file.

    Returns:
    - tuple: A tuple containing:
        - dict: A dictionary containing the SSBOND records and the corresponding ATOM records. The dictionary
          has the following structure:
            {
                "ssbonds": list of SSBOND records (str),
                "atoms": {
                    (chain_id, res_seq_num, atom_name): {
                        "x": x-coordinate (float),
                        "y": y-coordinate (float),
                        "z": z-coordinate (float)
                    },
                    ...
                },
                "pairs": [
                    {
                        "proximal": (chain_id1, res_seq_num1),
                        "distal": (chain_id2, res_seq_num2),
                        "chains": (chain_id1, chain_id2),
                        "phipsi": {
                            "proximal-1": {"N": [x, y, z], "C": [x, y, z]},
                            "proximal+1": {"N": [x, y, z], "C": [x, y, z]},
                            "distal-1": {"N": [x, y, z], "C": [x, y, z]},
                            "distal+1": {"N": [x, y, z], "C": [x, y, z]}
                        }
                    },
                    ...
                ]
            }
        - int: The number of SSBOND records found.
        - list: A list of error messages encountered during processing.
    """
    if not os.path.exists(input_pdb_file):
        return None

    ssbonds = []
    atom_list = {}
    errors = []
    pairs = []

    # Read the PDB file and collect SSBOND and ATOM records

    with open(input_pdb_file, "r") as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("SSBOND"):
            ssbonds.append(line)
        elif line.startswith("ATOM"):
            # Create a map to quickly find ATOM records by residue sequence number, chain ID, and atom name
            chain_id = line[21].strip()
            res_seq_num = line[22:26].strip()
            atom_name = line[12:16].strip()
            x = float(line[30:38].strip())
            y = float(line[38:46].strip())
            z = float(line[46:54].strip())
            key = (chain_id, res_seq_num, atom_name)
            atom_list[key] = {"x": x, "y": y, "z": z}
            if verbose:
                print(
                    f"Found ATOM record for chain {chain_id}, residue {res_seq_num}, atom {atom_name}"
                )

    # Extract the ATOM records corresponding to SSBOND
    ssbond_atom_list = {"ssbonds": ssbonds, "atoms": {}, "pairs": pairs}
    for ssbond in ssbonds:
        parts = ssbond.split()
        chain_id1 = parts[3]
        res_seq_num1 = parts[4]
        chain_id2 = parts[6]
        res_seq_num2 = parts[7]

        # Add the corresponding ATOM records to the ssbond_atom_list
        for atom_name in ["N", "CA", "C", "O", "CB", "SG"]:
            atom_record1 = atom_list.get((chain_id1, res_seq_num1, atom_name))
            atom_record2 = atom_list.get((chain_id2, res_seq_num2, atom_name))
            if atom_record1:
                ssbond_atom_list["atoms"][
                    (chain_id1, res_seq_num1, atom_name)
                ] = atom_record1
            else:
                errors.append(
                    f"Atom record not found for chain {chain_id1}, residue {res_seq_num1}, atom {atom_name}"
                )
                if verbose:
                    _logger.error(
                        f"Atom record not found for chain {chain_id1}, residue {res_seq_num1}, atom {atom_name}"
                    )

            if atom_record2:
                ssbond_atom_list["atoms"][
                    (chain_id2, res_seq_num2, atom_name)
                ] = atom_record2
            else:
                errors.append(
                    f"Atom record not found for chain {chain_id2}, residue {res_seq_num2}, atom {atom_name}"
                )
                if verbose:
                    _logger.error(
                        f"Atom record not found for chain {chain_id2}, residue {res_seq_num2}, atom {atom_name}"
                    )

        # Collect phi/psi related atoms
        def get_phipsi_atoms(chain_id, res_seq_num):
            phipsi_atoms = {}
            for offset in [-1, 1]:
                for atom_name in ["N", "C"]:
                    key = (chain_id, str(int(res_seq_num) + offset), atom_name)
                    atom_record = atom_list.get(key)
                    if atom_record:
                        if f"{res_seq_num}{offset}" not in phipsi_atoms:
                            phipsi_atoms[f"{res_seq_num}{offset}"] = {}
                        phipsi_atoms[f"{res_seq_num}{offset}"][atom_name] = [
                            atom_record["x"],
                            atom_record["y"],
                            atom_record["z"],
                        ]
                    else:
                        errors.append(
                            f"Atom record not found for chain {chain_id}, residue {str(int(res_seq_num) + offset)}, atom {atom_name}"
                        )
                        if verbose:
                            _logger.error(
                                f"Atom record not found for chain {chain_id}, residue {str(int(res_seq_num) + offset)}, atom {atom_name}"
                            )
            return phipsi_atoms

        phipsi = {
            "proximal-1": get_phipsi_atoms(chain_id1, res_seq_num1).get(
                f"{res_seq_num1}-1", {}
            ),
            "proximal+1": get_phipsi_atoms(chain_id1, res_seq_num1).get(
                f"{res_seq_num1}+1", {}
            ),
            "distal-1": get_phipsi_atoms(chain_id2, res_seq_num2).get(
                f"{res_seq_num2}-1", {}
            ),
            "distal+1": get_phipsi_atoms(chain_id2, res_seq_num2).get(
                f"{res_seq_num2}+1", {}
            ),
        }

        # Add the pair information to the pairs list
        pairs.append(
            {
                "proximal": (chain_id1, res_seq_num1),
                "distal": (chain_id2, res_seq_num2),
                "chains": (chain_id1, chain_id2),
                "phipsi": phipsi,
            }
        )

    return ssbond_atom_list, len(ssbonds), len(errors)

In [ ]:
from proteusPy import MODEL_DIR
from proteusPy.logger_config import get_logger

_logger = get_logger("Testing")


def Nload_disulfides_from_id(
    pdb_id: str,
    pdb_dir=MODEL_DIR,
    model_numb=0,
    verbose=False,
    quiet=True,
    dbg=False,
    cutoff=-1.0,
) -> DisulfideList:
    """
    Loads the Disulfides by PDB ID and returns a ```DisulfideList``` of Disulfide objects.
    Assumes the file is downloaded in the pdb_dir path.

    *NB:* Requires EGS-Modified BIO.parse_pdb_header.py from https://github.com/suchanek/biopython

    :param pdb_id: the name of the PDB entry.
    :param pdb_dir: path to the PDB files, defaults to MODEL_DIR - this is: PDB_DIR/good and are
    the pre-parsed PDB files that have been scanned by the DisulfideDownloader program.
    :param model_numb: model number to use, defaults to 0 for single structure files.
    :param verbose: print info while parsing
    :return: a list of Disulfide objects initialized from the file.

    Example:

    PDB_DIR defaults to os.getenv('PDB').
    To load the Disulfides from the PDB ID 5rsa we'd use the following:

    >>> from proteusPy import DisulfideList, load_disulfides_from_id
    >>> from proteusPy.ProteusGlobals import DATA_DIR
    >>> SSlist = DisulfideList([],'5rsa')
    >>> SSlist = load_disulfides_from_id('5rsa', pdb_dir=DATA_DIR, verbose=False)
    >>> SSlist
    [<Disulfide 5rsa_26A_84A, Source: 5rsa, Resolution: 2.0 Å>, <Disulfide 5rsa_40A_95A, Source: 5rsa, Resolution: 2.0 Å>, <Disulfide 5rsa_58A_110A, Source: 5rsa, Resolution: 2.0 Å>, <Disulfide 5rsa_65A_72A, Source: 5rsa, Resolution: 2.0 Å>]
    """
    import copy
    import os
    import warnings

    from proteusPy import Disulfide, extract_ssbonds_and_atoms, DisulfideList
    from Bio.PDB import PDBParser

    i = 1
    proximal = distal = -1
    chain1_id = chain2_id = ""

    resolution = -1.0

    parser = PDBParser(PERMISSIVE=True)

    # Biopython uses the Structure -> Model -> Chain hierarchy to organize
    # structures. All are iterable.
    structure_fname = os.path.join(pdb_dir, f"pdb{pdb_id}.ent")
    structure = parser.get_structure(pdb_id, file=structure_fname)
    model = structure[model_numb]

    if dbg:
        print(f"-> load_disulfide_from_id() - Parsing structure: {pdb_id}:")

    SSList = DisulfideList([], pdb_id, resolution)

    # list of tuples with (proximal distal chaina chainb)
    # ssbonds = parse_ssbond_header_rec(ssbond_dict)

    ssbond_atom_list, num_ssbonds, errors = extract_ssbonds_and_atoms(
        structure_fname, verbose=verbose
    )

    # with warnings.catch_warnings():
    if quiet:
        # _logger.setLevel(logging.ERROR)
        warnings.filterwarnings("ignore", category=BiopythonWarning)

    for pair in ssbond_atom_list["pairs"]:
        proximal, chain1_id = pair["proximal"]
        distal, chain2_id = pair["distal"]

        if not proximal.isnumeric() or not distal.isnumeric():
            mess = f" -> load_disulfides_from_id(): Cannot parse SSBond record (non-numeric IDs):\
            {pdb_id} Prox: {proximal} {chain1_id} Dist: {distal} {chain2_id}, ignoring."
            _logger.error(mess)
            continue
        else:
            proximal = int(proximal)
            distal = int(distal)

        if proximal == distal:
            if verbose:
                mess = f" -> load_disulfides_from_id(): SSBond record has (proximal == distal):\
                {pdb_id} Prox: {proximal} {chain1_id} Dist: {distal} {chain2_id}."
                _logger.info(mess)

        if verbose:
            _logger.info(
                f"SSBond: {i}: {pdb_id}: {proximal} {chain1_id} - {distal} {chain2_id}"
            )

        res = initialize_disulfide_from_coords(
            ssbond_atom_data,
            pdb_id,
            chain1_id,
            chain2_id,
            resolution,
            quiet=quiet,
        )
        if res:
            SSList.append(new_ss)
        i += 1

    if quiet:
        # _logger.setLevel(logging.WARNING)
        warnings.filterwarnings("ignore", category=BiopythonWarning)

    if cutoff > 0:
        SSList = SSList.filter_by_distance(cutoff)

    return copy.deepcopy(SSList)

In [ ]:
ss = load_disulfides_from_id("7o6v", pdb_dir="/Users/egs/PDB/good", verbose=True)


for ssbond in ss:
    print(f"Disulfide: {ssbond}, Ca: {ssbond.ca_distance}")

In [ ]:
def find_disulfides(pdb, id) -> DisulfideList:
    """
    Find disulfide in pdb object.
    """

    indices = pdb.SSDict[id]
    print(f"indices: {indices}")
    res = DisulfideList([], id)
    sslist = pdb.SSList
    for ind in indices:
        print(f"ind: {ind} sslist[ind]: {sslist[ind]}")
        res.append(sslist[ind])
    return res

In [ ]:
# PDB_SS['4yys'] return a list of SS

sslist = find_disulfides(pdb, "4yys")
sslist

In [ ]:
def find_null_pdb_indices(pdb, limit=1000):
    """
    Loops over pdb entries from 0 to limit (default 1000) and checks each entry for null.
    Returns a list of indices with null entries.
    """
    null_indices = []
    ids = pdb.IDList
    for i in ids:
        if len(pdb[i]) == 0:
            null_indices.append(i)
    return null_indices


def find_null_pdb_keys(pdb, limit=1000):
    """
    Loops over pdb entries from 0 to limit (default 1000) and checks each entry for null.
    Returns a list of indices with null entries.
    """
    null_indices = []
    ssdict = pdb.SSDict
    for i in ssdict:
        if len(ssdict[i]) == 0:
            null_indices.append(i)
    return null_indices

In [ ]:
missing = find_null_pdb_indices(pdb)
len(missing)
missing

In [ ]:
bad = []

for id in missing:
    res = load_disulfides_from_id(id, verbose=True)
    if len(res) == 0:
        print(f"ID {id} is missing disulfides")
    else:
        for ss in res:
            if ss.ca_distance > 8.0:
                print(f"ID {id} has a long disulfide: {ss}")
                bad.append(ss.name)


bad

In [ ]:
pdb["6vkk_845A_845C"]

In [ ]:
idlist = pdb.IDList
"6vkk" in idlist

In [ ]:
def find_disulfides(pdb, id) -> DisulfideList:
    """
    Find disulfide in pdb object.
    """

    indices = pdb.SSDict[id]
    # print(f"indices: {indices}")
    res = DisulfideList([], id)
    sslist = pdb.SSList
    for ind in indices:
        # print(f"ind: {ind} sslist[ind]: {sslist[ind]}")
        res.append(sslist[ind])
    return res

In [ ]:
# PDB_SS['4yys'] return a list of SS

sslist = find_disulfides(pdb, "4wmy")
sslist

In [ ]:
pdb["4wmy"]

In [ ]:
find_null_pdb_indices(pdb)

In [ ]:
sslist, xchain = prune_extra_ss(wym)
sslist

In [ ]:
from proteusPy import remove_duplicate_ss

pruned = remove_duplicate_ss(wym)
pruned

In [ ]:
def find_string_in_list(target_string, list_of_strings):
    """
    Searches for a target string in a list of strings and returns the index if found.
    Returns -1 if the target string is not found.
    """
    try:
        return list_of_strings.index(target_string)
    except ValueError:
        return -1

In [ ]:
find_string_in_list("4wmy", pdb.IDList)
pdb["4wmy"]

In [ ]:
wmy = pdb["4wmy"]
wmy

In [ ]:
ss1 = pdb["4yys_22A_65A"]
ss2 = pdb["4yys_22B_65B"]
ss1 == ss2

In [ ]:
ss1.pprint_all()

In [ ]:
ss2.pprint_all()

In [ ]:
def remove_duplicate_ss(sslist: DisulfideList) -> DisulfideList:
    pruned = []
    for ss in sslist:
        if ss not in pruned:
            pruned.append(ss)
    return pruned

In [ ]:
yys = pdb["4wmy"]
yys

In [ ]:
pruned = remove_duplicate_ss(yys)
pruned

In [ ]:
def compare_dihedrals(self, other) -> float:
    """
    Compare the Disulfide object's dihedrals to another Disulfide object's dihedrals.

    :param other: Disulfide object to compare to
    :return: The length of the difference of the two sets of dihedral angles
    :raises TypeError: If the input is not a Disulfide object
    """
    import numpy
    from Bio.PDB import Vector

    def cmp_vec(v1: Vector, v2: Vector) -> float:
        "Return the length of the difference between the two vectors."
        _diff = v2 - v1
        _len = _diff.norm()
        return _len

    if isinstance(other, Disulfide):
        dihed1 = Vector(self.torsion_array)
        dihed2 = Vector(other.torsion_array)
        return cmp_vec(dihed1, dihed2)
    else:
        raise TypeError("Input must be a Disulfide object.")

In [ ]:
def Torsion_RMS(first, other) -> float:
    """
    Calculate the RMS distance between the dihedral angles of self and another Disulfide.
    :param other: Comparison Disulfide
    :return: RMS distance (deg)
    """
    import math

    # Get internal coordinates of both objects
    ic1 = first.torsion_array
    ic2 = other.torsion_array

    # Compute the sum of squared differences between corresponding internal coordinates
    totsq = sum((p1 - p2) ** 2 for p1, p2 in zip(ic1, ic2))
    # Compute the mean of the squared distances
    totsq /= len(ic1)

    # Take the square root of the mean to get the RMS distance
    return math.sqrt(totsq)

In [ ]:
ss1 = pdb[0]
ss1

In [ ]:
Torsion_RMS(ss1, ss1)